# Telecom Revenue Assurance AI Model (Fine-Tuned BERT)
Author: Fatih E. NAR



In [ ]:
# Install Necessary Libraries
!pip install -r requirements.txt
!pip install transformers datasets torch scikit-learn accelerate -U
!pip install transformers datasets accelerate peft bitsandbytes

In [ ]:
# Import Libraries
import pandas as pd
import torch
import numpy as np
import lzma
import shutil
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb

# Extract the .xz file
with lzma.open('data/telecom_revass_data.csv.xz', 'rb') as f_in:
    with open('data/telecom_revass_data.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Check if MPS (Metal Performance Shaders) or CUDA is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = torch.device("cpu")


print(f'Using device: {device}')

# Load your dataset
df = pd.read_csv('data/telecom_revass_data.csv')

# Convert data to text format (example)
df['text'] = df.apply(lambda row: f"Call Duration: {row['Call_Duration']}, Data Usage: {row['Data_Usage']}, SMS Count: {row['Sms_Count']}, Roaming Indicator: {row['Roaming_Indicator']}, Mobile Wallet Use: {row['MobileWallet_Use']}, Plan Type: {row['Plan_Type']}, Cost: {row['Cost']}, Cellular Location Distance: {row['Cellular_Location_Distance']}, Last Time Pin Used: {row['Last_Time_Pin_Used']}, Avg Call Duration: {row['Avg_Call_Duration']}, Avg Data Usage: {row['Avg_Data_Usage']}, Avg Cost: {row['Avg_Cost']}", axis=1)

# Split the data
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['Fraud'])

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['text', 'Fraud']])
test_dataset = Dataset.from_pandas(test_df[['text', 'Fraud']])

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Apply LoRA to the model
#config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, bias="none")
config = LoraConfig(r=4, lora_alpha=8, lora_dropout=0.2, bias="none")
model = get_peft_model(model, config)

# Enable 8-bit precision by using the 8-bit optimizer < Enable when using with NVIDIA
optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=5e-5)


In [ ]:
# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column('Fraud', 'labels')
test_dataset = test_dataset.rename_column('Fraud', 'labels')

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,  # Accumulate gradients
    fp16=True,  # Enable mixed precision training  < Enable when using with NVIDIA
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=10,
)

# Define custom compute_metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = (pred == labels).mean()
    return {'accuracy': accuracy}

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    #train_dataset=train_dataset,
    #eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset.select(range(1000)),  # Use a subset for debugging
    eval_dataset=test_dataset.select(range(200)),  # Use a subset for debugging
    tokenizer=tokenizer,
    optimizers=(optimizer, None),  # Custom optimizer  < Enable when using with NVIDIA
)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(results)

In [ ]:
# Save the fine-tuned model
model.save_pretrained('models/fine-tuned-bert-perf-revass')
tokenizer.save_pretrained('models/fine-tuned-bert-perf-revass')